<a href="https://colab.research.google.com/github/vit050587/Recommendation-systems/blob/master/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install implicit

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k
import os, sys

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_sistems/lesson02/webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [7]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 3.06 s, sys: 5.63 ms, total: 3.07 s
Wall time: 3.07 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[976059, 93285, 1363945, 15596225, 1031867]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1136926, 9835931, 15717056, 12692072, 7173292]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
#функция для получения весов товаров в зависимости от объёма продаж в денежном эквиваленте
def get_items_weights(df):
    total_sales=df['sales_value'].sum()
    items_weights=df.groupby('item_id').agg({'sales_value':'sum'}).reset_index().rename(columns={'sales_value':'weight'})
    items_weights['weight']=items_weights['weight'].apply(lambda x: x/total_sales)
    return items_weights

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    weights=np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False) #используем параметр p метода choice в который передадим веса товаров
    
    
    return recs.tolist()

In [10]:
%%time

# your_code
items_weights=get_items_weights(data_train)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 2.55 s, sys: 22 ms, total: 2.57 s
Wall time: 2.58 s


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[976059, 93285, 1363945, 15596225, 1031867]","[1083721, 10149640, 7189022, 6534178, 9676822]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1136926, 9835931, 15717056, 12692072, 7173292]","[948650, 941468, 6534178, 1076875, 1103019]"


In [11]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [13]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

In [14]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [15]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.0213:weighted_random_recommendation


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [17]:
#result = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommendation_sistems/lesson02/webinar_2/predictions_basic.csv')
#result.head(2)

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Посчитаем случайный на топ_5000 товаров

In [16]:
#функция для получения Топ 5000 товаров по количеству проданного(можно варьировать какой Топ получать и по какому показателю)
def get_top(df, column='quantity', top=5000):
    top_df=df.groupby('item_id').agg({f'{column}':'sum'}).reset_index().sort_values(column, ascending=False).head(5000).item_id.tolist()
    return top_df

In [17]:
# your_code
%%time

items = get_top(data_train)

result['top5000_random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 1.33 s, sys: 24.9 ms, total: 1.35 s
Wall time: 1.39 s


,user_id,actual,random_recommendation,weighted_random_recommendation,top5000_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[976059, 93285, 1363945, 15596225, 1031867]","[1083721, 10149640, 7189022, 6534178, 9676822]","[1090793, 1034686, 913210, 945909, 5587656]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1136926, 9835931, 15717056, 12692072, 7173292]","[948650, 941468, 6534178, 1076875, 1103019]","[904422, 850058, 1055412, 901747, 1102416]"


Посчитаем качество случайной рекомендации на товарах из Топ-5000 по количеству проданного товара и по сумме продаж

In [18]:
%%time

items = get_top(data_train, column='sales_value')

result['top5000_sales_random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 1.04 s, sys: 33.9 ms, total: 1.07 s
Wall time: 1.09 s


,user_id,actual,random_recommendation,weighted_random_recommendation,top5000_random_recommendation,top5000_sales_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[976059, 93285, 1363945, 15596225, 1031867]","[1083721, 10149640, 7189022, 6534178, 9676822]","[1090793, 1034686, 913210, 945909, 5587656]","[1042907, 866227, 1065979, 830304, 849018]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1136926, 9835931, 15717056, 12692072, 7173292]","[948650, 941468, 6534178, 1076875, 1103019]","[904422, 850058, 1055412, 901747, 1102416]","[1066893, 918598, 1128399, 889989, 891516]"


In [19]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.0213:weighted_random_recommendation
0.0073:top5000_random_recommendation
0.0055:top5000_sales_random_recommendation


**Видим, что качество случайной рекомендации возросло, но не на столько, как при использовании взвешенной случайной рекомендации**

посчитаем качество ItemItemRecommender, выбирая число соседей K

In [20]:
top_5000= get_top(data_train)

In [21]:
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [22]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [23]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Подбираем количество соседей:

In [ ]:
neighbors=np.arange(1,21)
fltrs=[None,[itemid_to_id[6666]]]
max_score, m_neighbor, m_fltr = 0,0,0
res_dict={'no_fltr':[[],[]],'fltr':[[],[]]}
for neighbor in neighbors:
    for fltr in fltrs:
        current_key='fltr' if fltr else 'no_fltr'
        model = ItemItemRecommender(K=neighbor, num_threads=4)
        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)
        
        result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=fltr, 
                                                recalculate_user=True)
                                                             ])
        score=round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(),4)
        res_dict[current_key][0].append(score)
        res_dict[current_key][1].append(neighbor)
        
        if score> max_score:
            max_score, m_neighbor, m_fltr=score, neighbor, 'filtered by 6666' if fltr else 'non filtered'
print(f'Лучший скор: {max_score}, K_neighbors: {m_neighbor}, условие фильтрации: {m_fltr}')       

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(res_dict['fltr'][1], res_dict['fltr'][0], label='filtered by 6666')
plt.plot(res_dict['no_fltr'][1], res_dict['no_fltr'][0], label='non filtered')
plt.xlabel('N-neighbor')
plt.ylabel('precision_at_k')
plt.legend()
plt.show()